In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from scipy.ndimage.interpolation import shift
import xgboost as xgb
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

/var/folders/lj/lmb91jvd0zb9d59rd05y9xhw0000gn/T/ipykernel_2023/962963114.py:3: DeprecationWarning: Please use `shift` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import shift


In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]
y = y.astype(np.uint8)
X = np.array(X)

X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

/opt/homebrew/Caskroom/miniconda/base/envs/datasci/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
print(X_test.shape)

(10000, 784)


In [4]:
# 数据增强函数：对角、水平、垂直位移
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [5]:
print("Creating Augmented Dataset...")

# 初始化增强数据集
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

# 应用水平和垂直位移
for dx, dy in tqdm(((1, 0), (-1, 0), (0, 1), (0, -1)), desc="Applying horizontal and vertical shifts"):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

# 应用对角线位移
for dx, dy in tqdm([(1, 1), (-1, -1), (1, -1), (-1, 1)], desc="Applying diagonal shifts"):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

# 打乱增强数据集
shuffle_idx = np.random.permutation(len(X_train_augmented))
X_train_augmented = np.array(X_train_augmented)[shuffle_idx]
y_train_augmented = np.array(y_train_augmented)[shuffle_idx]

print("Creating Augmented Dataset completed")

Creating Augmented Dataset...


Applying diagonal shifts: 100%|██████████| 4/4 [00:15<00:00,  4.00s/it]


Creating Augmented Dataset completed


In [6]:
# 训练 XGBoost 模型，加入早期停止
print("Training XGBoost model...")
xgb_clf = xgb.XGBClassifier(objective='multi:softprob', num_class=10, n_estimators=500)
xgb_clf.fit(X_train_augmented, y_train_augmented, 
            eval_set=[(X_test, y_test)], 
            eval_metric="mlogloss", 
            early_stopping_rounds=10,  
            # 如果在10轮迭代内验证集上的性能没有改善，则停止训练
            verbose=True)

# 预测和评估
y_pred = xgb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Training XGBoost model...


/opt/homebrew/Caskroom/miniconda/base/envs/datasci/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/datasci/lib/python3.9/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.47033
[1]	validation_0-mlogloss:1.15243
[2]	validation_0-mlogloss:0.94140
[3]	validation_0-mlogloss:0.79210
[4]	validation_0-mlogloss:0.67744
[5]	validation_0-mlogloss:0.58644
[6]	validation_0-mlogloss:0.51597
[7]	validation_0-mlogloss:0.45912
[8]	validation_0-mlogloss:0.41017
[9]	validation_0-mlogloss:0.36911
[10]	validation_0-mlogloss:0.33536
[11]	validation_0-mlogloss:0.30646
[12]	validation_0-mlogloss:0.28288
[13]	validation_0-mlogloss:0.26356
[14]	validation_0-mlogloss:0.24624
[15]	validation_0-mlogloss:0.23346
[16]	validation_0-mlogloss:0.22041
[17]	validation_0-mlogloss:0.20838
[18]	validation_0-mlogloss:0.19788
[19]	validation_0-mlogloss:0.19007
[20]	validation_0-mlogloss:0.18317
[21]	validation_0-mlogloss:0.17540
[22]	validation_0-mlogloss:0.16947
[23]	validation_0-mlogloss:0.16332
[24]	validation_0-mlogloss:0.15712
[25]	validation_0-mlogloss:0.15300
[26]	validation_0-mlogloss:0.14757
[27]	validation_0-mlogloss:0.14345
[28]	validation_0-mlogloss:0.1